In [72]:
import numpy as np
import random
import copy

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from dataset import TimeSeriesDataset, TimeSeriesDatasetConcat
from martins.complex_transformer import ComplexTransformer

In [25]:
# Parameters
parser = argparse.ArgumentParser(description='JDA Time series adaptation')
parser.add_argument("--data-path", type=str, default="/projects/rsalakhugroup/complex/domain_adaptation", help="dataset path")
parser.add_argument("--task", type=str, help='3A or 3E')
parser.add_argument('--batch_size', type=int, default=256, help='batch size')
parser.add_argument('--epochs_gan', type=int, default=50, help='number of epochs for training GAN')
parser.add_argument('--epochs', type=int, default=50, help='number of epochs')
parser.add_argument('--lr_gan', type=float, default=1e-4, help='learning rate for adversarial')
parser.add_argument('--lr_clf', type=float, default=1e-4, help='learning rate for classification')
parser.add_argument('--seed', type=int, help='manual seed')
parser.add_argument('--gap', type=int, help='gap: Generator train GAP times, discriminator train once')
parser.add_argument('--model', type=str, default="fnn", help='manual seed')
parser.add_argument('--file', type=str, default="processed_file_{}_train_test_20.pkl", help='which file(data split) to perform')

args = parser.parse_args()
device = torch.device("cuda:0")

# seed
if args.seed is None:
    args.seed = random.randint(1, 10000)
random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)
cudnn.deterministic = True
torch.backends.cudnn.deterministic = True

print(args.model)

NameError: name 'argparse' is not defined

In [29]:
class Generator(nn.Module):
    def __init__(self, feature_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            #nn.Tanh()
        ) 

    def forward(self, x):
        # x: [bs, seq, init_size (small)]
        return self.net(x)


In [30]:
class Discriminator(nn.Module):
    def __init__(self, feature_dim, d_out):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(

            nn.Linear(feature_dim, feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),
        ) 
        self.fc = nn.Linear(3200, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        # x: [bs, seq, feature_dim]
        x = self.net(x)
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        out = self.sigmoid(self.fc(x))
        return out


In [31]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('LayerNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [74]:
def get_source_dict(file_path, num_class):
    '''
    output:
        {class: [data]}
    '''
    data_ = np.load(file_path, allow_pickle=True)
    train_data = data_['tr_data']
    train_lbl = data_['tr_lbl']
    data_dict = get_class_data_dict(train_data, train_lbl, num_class)
    
    return data_dict

def get_target_dict(file_path, num_class, lbl_percentage, seed=0):
    '''
    split target domain data
    
    output:
        with label:
            {class: [data]}
        without label:
            [data], [lbl]
            
    '''
    data_ = np.load(file_path, allow_pickle=True)
    train_data = data_['te_data']
    train_lbl = data_['te_lbl']
    
    np.random.seed(seed)
    index = np.random.permutation(train_data.shape[0])
    train_data = train_data[index]
    train_lbl = train_lbl[index]
    
    unlabel_train_data = []
    unlabel_train_lbl = []
    
    with_label = {i:[] for i in num_class}
    for i in range(train_lbl.shape[0]):
        if len(with_label[train_lbl[i]]) < int(lbl_percentage*train_lbl.shape[0]/num_class):
            with_label[train_lbl[i]].append(train_data[i])
        else:
            unlabel_train_data.append(train_data[i])
            unlabel_train_lbl.append(train_lbl[i])
    
    return with_label, (unlabel_train_data, unlabel_train_lbl)

def get_class_data_dict(data, lbl, num_class):
    '''
    construct a dict {label: data}  
    '''
    lbl_not_one_hot = np.argmax(lbl)
    result = {i:[] for i in num_class}
    for i in range(lbl_not_one_hot.shape[0]):
        result[lbl_not_one_hot[i]].append(data[i])
        
    for i in result:
        result[i] = np.array(result[i])
        
    return result

def get_batch_source_data_on_class(class_dict, num_per_class):
    '''
    get batch from source data given a required number of sample per class
    '''
    batch_x = []
    batch_y = []
    for key, value in class_dict:
        index = random.sample(range(len(value)), num_per_class)
        batch_x.append(value[index])
        batch_y.append([key] * num_per_class)
        
    return np.array(batch_x), np.array(batch_y)

def get_batch_target_data_on_class(real_dict, pesudo_dict, unlabel_data, num_per_class, compromise=3, real_weight=1, pesudo_weight=0.01):
    '''
    get batch from target data given a required number of sample per class
    '''
    batch_x = []
    batch_y = []
    batch_real_or_pesudo = []
    for key in real_dict:
        real_num = len(real_dict[key])
        pesudo_num = len(pesudo_dict[key])
        num_in_class = real_num + pesudo_num
        
        if num_in_class < num_per_class:
            # if totoal number sample in this class is less than the required number of sample
            # then fetch the remainding data randomly from the unlabeled set with a compromise
            num_fetch_unlabeled = (num_in_class - num_per_class) * compromise
            index = random.sample(range(unlabel_data.shape[0]), num_fetch_unlabeled)
            batch_x.append(unlabel_data[index])
            batch_y.append([key] * num_fetch_unlabeled)
            batch_real_or_pesudo.append([pesudo_weight] * num_fetch_unlabeled)
            
            batch_x.append(real_dict[key])
            batch_real_or_pesudo.append([real_weight] * real_num)
            batch_x.append(pesudo_dict[key])
            batch_real_or_pesudo.append([pesudo_weight] * pesudo_num)
            batch_y.append([key] * num_in_class)
            
        else:
            index = random.sample(range(num_in_class), num_per_class)
            index_in_real = []
            index_in_pesudo = []
            for i in index:
                if i >= real_num:
                    index_in_pesudo.append(i-real_num)
                else:
                    index_in_real.append(i)
                    
            batch_x.append(real_dict[key][index_in_real])
            batch_real_or_pesudo.append([real_weight] * len(index_in_real))
            batch_x.append(pesudo_dict[key][index_in_pesudo,:])
            batch_real_or_pesudo.append([pesudo_weight] * len(index_in_pesudo))
            batch_y.append([key] * num_per_class)
    
    return np.array(batch_x), np.array(batch_y), np.array(batch_real_or_pesudo)

# Main

Model architecture

encoder: feature extractor
CNet:    Classifier
DNet_global:    global Discriminator
DNet_local:    class-wise Discriminator
GNet:    Generator (Adaptor)

In [ ]:
# source_unlabel_set = TimeSeriesDataset(root_dir=args.data_path, file_name=args.file.format(args.task), train=True)
# target_unlabel_set = TimeSeriesDataset(root_dir=args.data_path, file_name=args.file.format(args.task), train=False)
# source_unlabel_loader = DataLoader(source_unlabel_set, batch_size=args.batch_size, shuffle=True)
# target_unlabel_loader = DataLoader(target_unlabel_set, batch_size=args.batch_size, shuffle=True)


# TODO: modify argument
source_dict = get_source_dict(file_path, num_class)
target_dict, (target_unlabel_x, target_unlabel_y) = get_target_dict(file_path, num_class, lbl_percentage, seed=0)

# TODO: restore classifer
seq_len = 10 
feature_dim = 160
d_out = 50 if args.task == "3A" else 65

encoder = ComplexTransformer(layers=1, 
                           time_step=seq_len, 
                           input_dim=feature_dim, 
                           hidden_size=512, 
                           output_dim=512, 
                           num_heads=8,
                           out_dropout=0.5)
encoder.to(device)
encode


# initialize GAN
feature_dim_joint = 2 * feature_dim
DNet_global = Discriminator(feature_dim=feature_dim_joint, d_out=d_out).to(device)
DNet_local = Discriminator(feature_dim=feature_dim_joint, d_out=d_out).to(device)
GNet = Generator(feature_dim=feature_dim_joint).to(device)
DNet_global.apply(weights_init)
DNet_local.apply(weights_init)
GNet.apply(weights_init)
optimizerD_global = torch.optim.Adam(DNet_global.parameters(), lr=args.lr_gan)
optimizerD_local = torch.optim.Adam(DNet_local.parameters(), lr=args.lr_gan)
optimizerG = torch.optim.Adam(GNet.parameters(), lr=args.lr_gan)

# TODO: add global & local loss
criterion_gan = nn.BCELoss()
schedulerD_global = torch.optim.lr_scheduler.StepLR(optimizerD_global, step_size=30, gamma=0.1)
schedulerD_local = torch.optim.lr_scheduler.StepLR(optimizerD_local, step_size=30, gamma=0.1)
schedulerG = torch.optim.lr_scheduler.StepLR(optimizerG, step_size=30, gamma=0.1)

feature_dim_joint = 2 * feature_dim
joint_set = TimeSeriesDatasetConcat(root_dir=args.data_path, file_name=args.file.format(args.task), seed=args.seed)
joint_loader = DataLoader(joint_set, batch_size=args.batch_size, shuffle=True)
total_error_D = total_error_G = 0

source_mean = joint_set.tr_data_mean
target_mean = joint_set.te_data_mean
source_std = joint_set.tr_data_std
target_std = joint_set.te_data_std

for epoch in range(args.epochs_gan):
    # get pesudo label
    correct_target
    for batch in range(math.ceil(target_unlabel_x.shape[0],args.batch_size)):
        target_unlabel_x_batch = target_unlabel_x[:batch*args.batch_size].to(device).float()
        pred = classifier(target_unlabel_x_batch)
        correct_target += (pred.argmax(-1) == y.argmax(-1)).sum().item()

    target_pesudo_y = classifier(target_unlabel_x)
    pesudo_dict = get_class_data_dict(target_unlabel_x, target_pesudo_y, num_class)
target_pesudo_y
    

    # fake: target domain
    # real: source domain
    total_error_D = total_error_G = 0
    for batch_id, (source_x, target_x) in enumerate(joint_loader):
        batch_size = target_x.shape[0]
        target_x = target_x.reshape(batch_size, seq_len, feature_dim_joint)
        source_x = source_x.reshape(batch_size, seq_len, feature_dim_joint)

        # Data Normalization
        target_x = (target_x - target_mean) / target_std
        source_x = (source_x - source_mean) / source_std

        """Update D Net"""
        # train with source domain
        DNet_global.zero_grad()
        source_data = source_x.to(device).float()
        label = torch.full((batch_size,), real_label, device=device)
        output = DNet_global(source_data).view(-1)
        #print(output.mean().item())
        errD_global_source = criterion_gan(output, label)
        errD_global_source.backward()

        # train with target domain
        target_data = target_x.to(device).float()
        fake = GNet(target_data)
        #print(fake)
        label.fill_(fake_label)
        output = DNet_global(fake.detach()).view(-1)
        #print(output.mean().item())
        errD_global_target = criterion_gan(output, label)
        errD_global_target.backward()
        total_error_D_global += (errD_global_source + errD_global_target).item()
        
        # TODO: gap????
        if batch_id % args.gap == 0:
            optimizerD_global.step()

        """Update G Network"""
        GNet.zero_grad()
        label.fill_(real_label) # fake labels are real for generator cost
        output = DNet_global(fake).view(-1)
        #print(output.mean().item())
        #print()
        errG = criterion_gan(output, label)
        errG.backward()
        optimizerG.step()
        total_error_G += errG.item()

    # TODO: batch num???
    for batch in range(math.ceil(/args.batch_size)):
        target_x, target_y, target_weight = get_batch_target_data_on_class(target_dict, pesudo_dict, num_per_class)
        source_x, source_y = get_batch_source_data_on_class(source_dict, num_per_class)
        
        # Data Normalization
        target_x = (target_x - target_mean) / target_std
        source_x = (source_x - source_mean) / source_std
        
        """Update D Net"""
        # train with source domain
        DNet_local.zero_grad()
        source_data = source_x.to(device).float()
        label = torch.full((batch_size,), real_label, device=device)
        output = DNet_local(source_data).view(-1)
        #print(output.mean().item())
        errD_local_source = criterion_gan(output, label)
        errD_local_source.backward()

        # train with target domain
        target_data = target_x.to(device).float()
        fake = GNet(target_data)
        #print(fake)
        label.fill_(fake_label)
        output = DNet_local(fake.detach()).view(-1)
        #print(output.mean().item())
        errD_local_target = criterion_gan(output, label)
        errD_local_target.backward()
        total_error_D_local += (errD_local_source + errD_local_target).item()
        
        # TODO: gap????
        if batch_id % args.gap == 0:
            optimizerD_local.step()

        """Update G Network"""
        GNet.zero_grad()
        label.fill_(real_label) # fake labels are real for generator cost
        output = DNet_local(fake).view(-1)
        #print(output.mean().item())
        #print()
        errG = criterion_gan(output, label)
        errG.backward()
        optimizerG.step()
        total_error_G += errG.item()
        
        
    total_error_G = total_error_G/2
    schedulerD_global.step()
    schedulerD_local.step()
    schedulerG.step()
    print('Epoch: %i, total loss: %f, G loss: %f, D_global loss: %f, D_local loss: %f'%(
        epoch, total_error_D_local+total_error_D_global+total_error_G, total_error_D, total_error_G_global, local_error_G_global))



In [ ]:
source_unlabel_set = TimeSeriesDataset(root_dir=args.data_path, file_name=args.file.format(args.task), train=True)
target_unlabel_set = TimeSeriesDataset(root_dir=args.data_path, file_name=args.file.format(args.task), train=False)
source_unlabel_loader = DataLoader(source_unlabel_set, batch_size=args.batch_size, shuffle=True)
target_unlabel_loader = DataLoader(target_unlabel_set, batch_size=args.batch_size, shuffle=True)

# TODO: modify argument
source_dict = get_source_dict(file_path, num_class)
target_dict, (target_unlabel_x, target_unlabel_y) = get_target_dict(file_path, num_class, lbl_percentage, seed=0)

seq_len = 10 
feature_dim = 160
d_out = 50 if args.task == "3A" else 65

# Model architecture

# encoder: feature extractor
# CNet:    Classifier
# DNet:    Discriminator
# GNet:    Generator (Adaptor)
encoder = ComplexTransformer(layers=1, 
                           time_step=seq_len, 
                           input_dim=feature_dim, 
                           hidden_size=512, 
                           output_dim=512, 
                           num_heads=8,
                           out_dropout=0.5)
encoder.to(device)


if args.model == "fnn":
    pass
elif args.model == "gan":
    feature_dim_joint = 2 * feature_dim
    DNet_global = Discriminator(feature_dim=feature_dim_joint, d_out=d_out).to(device)
    DNet_local = Discriminator(feature_dim=feature_dim_joint, d_out=d_out).to(device)
    GNet = Generator(feature_dim=feature_dim_joint).to(device)
    DNet_global.apply(weights_init)
    DNet_local.apply(weights_init)
    GNet.apply(weights_init)
    optimizerD_global = torch.optim.Adam(DNet_global.parameters(), lr=args.lr_gan)
    optimizerD_local = torch.optim.Adam(DNet_local.parameters(), lr=args.lr_gan)
    optimizerG = torch.optim.Adam(GNet.parameters(), lr=args.lr_gan)
    
    # TODO: add global & local loss
    criterion_gan = nn.BCELoss()
    schedulerD_global = torch.optim.lr_scheduler.StepLR(optimizerD_global, step_size=30, gamma=0.1)
    schedulerD_local = torch.optim.lr_scheduler.StepLR(optimizerD_local, step_size=30, gamma=0.1)
    schedulerG = torch.optim.lr_scheduler.StepLR(optimizerG, step_size=30, gamma=0.1)

    real_label = 0.99 # target domain
    fake_label = 0.01 # source domain

    # Generator: target to source

    
    # a joint dataset for training GAN
    joint_set = TimeSeriesDatasetConcat(root_dir=args.data_path, file_name=args.file.format(args.task), seed=args.seed)
    joint_loader = DataLoader(joint_set, batch_size=args.batch_size, shuffle=True)
    total_error_D = total_error_G = 0

    real_mean = joint_set.tr_data_mean
    fake_mean = joint_set.te_data_mean
    real_std = joint_set.tr_data_std
    fake_std = joint_set.te_data_std
    for epoch in range(args.epochs_gan):
        # fake: target domain
        # real: source domain
        total_error_D = total_error_G = 0
        for batch_id, (x_real, x_fake) in enumerate(joint_loader):
            batch_size = x_fake.shape[0]
            x_fake = x_fake.reshape(batch_size, seq_len, feature_dim_joint)
            x_real = x_real.reshape(batch_size, seq_len, feature_dim_joint)

            # Data Normalization
            x_fake = (x_fake - fake_mean) / fake_std
            x_real = (x_real - real_mean) / real_std

            """Update D Net"""
            # train with real(source domain)
            DNet.zero_grad()
            real_data = x_real.to(device).float()
            label = torch.full((batch_size,), real_label, device=device)
            output = DNet(real_data).view(-1)
            #print(output.mean().item())
            errD_real = criterion_gan(output, label)
            errD_real.backward()

            # train with fake(target domain)
            fake_data = x_fake.to(device).float()
            fake = GNet(fake_data)
            #print(fake)
            label.fill_(fake_label)
            output = DNet(fake.detach()).view(-1)
            #print(output.mean().item())
            errD_fake = criterion_gan(output, label)
            errD_fake.backward()
            total_error_D += (errD_real + errD_fake).item()
            if batch_id % args.gap == 0:
                optimizerD.step()

            """Update G Network"""
            GNet.zero_grad()
            label.fill_(real_label) # fake labels are real for generator cost
            output = DNet(fake).view(-1)
            #print(output.mean().item())
            #print()
            errG = criterion_gan(output, label)
            errG.backward()
            optimizerG.step()
            total_error_G += errG.item()
        schedulerD.step()
        schedulerG.step()
        print(total_error_D + total_error_G, total_error_D, total_error_G)
    print("Gan trainig finished") 


CNet = FNN(d_in=feature_dim * 2 * seq_len, d_h=500, d_out=d_out, dp=0.5).to(device)
params = list(encoder.parameters()) + list(CNet.parameters())

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(params, lr=args.lr_clf)

scheduler = ReduceLROnPlateau(optimizer, 'min')
# Encoding by complex transformer
x_mean_tr = training_set.data_mean
x_std_tr = training_set.data_std

x_mean_te = test_set.data_mean
x_std_te = test_set.data_std
best_acc_train = best_acc_test = 0
for epoch in range(args.epochs):
    """Training"""
    correct_train = 0
    total_bs_train = 0 # total batch size
    train_loss = 0
    for batch_id, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        batch_size = x.shape[0]
        #normalize data
        x = (x - x_mean_tr) / x_std_tr
        # take the real and imaginary part out
        real = x[:,:,0].reshape(batch_size, seq_len, feature_dim).float().to(device)
        imag = x[:,:,1].reshape(batch_size, seq_len, feature_dim).float().to(device)
        real, imag = encoder(real, imag)
        pred = CNet(torch.cat((real, imag), -1).reshape(x.shape[0], -1)) 
        loss = criterion(pred, y.argmax(-1))
        #print(pred.argmax(-1), y.argmax(-1))
        correct_train += (pred.argmax(-1) == y.argmax(-1)).sum().item()
        total_bs_train += y.shape[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.shape[0]
    train_acc = float(correct_train) / total_bs_train
    train_loss = train_loss / total_bs_train
    best_acc_train = max(best_acc_train, train_acc)

    """Testing"""
    correct_test = 0
    total_bs_test = 0
    test_loss = 0
    for batch_id, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        x = (x - x_mean_te) / x_std_te
        batch_size = x.shape[0]

        CNet.eval()
        encoder.eval()

        with torch.no_grad():
            #print(x.mean(), x.std())
            if args.model == "gan":
                # feed into Generator (target to source)
                print(x.mean().item(), x.std().item())
                x = x.reshape(batch_size, seq_len, feature_dim * 2).float()
                GNet.eval()
                x = GNet(x)
                print(x.mean().item(), x.std().item())
                print()
                x = x.reshape(batch_size, seq_len * feature_dim, 2) # reshape back
            print(x.mean(), x.std())
            real = x[:,:,0].reshape(x.shape[0], seq_len, feature_dim).float().to(device)
            imag = x[:,:,1].reshape(x.shape[0], seq_len, feature_dim).float().to(device)
            real, imag = encoder(real, imag)
            #print(real.mean(), real.std())
            #print()
            #print(imag.mean(), imag.std())
            #print("***********************")
            pred = CNet(torch.cat((real, imag), -1).reshape(x.shape[0], -1)) 
            #print(pred)
            loss = criterion(pred, y.argmax(-1))
            correct_test += (pred.argmax(-1) == y.argmax(-1)).sum().item()
            total_bs_test += y.shape[0]
            test_loss += loss.item() * x.shape[0]
    test_acc = float(correct_test) / total_bs_test
    test_loss = test_loss / total_bs_test

    best_acc_test = max(best_acc_test, test_acc)


    print("train loss", train_loss, "test_loss", test_loss)
    print("train acc", train_acc, "test_acc", test_acc)

print(best_acc_train, best_acc_test)
print(args.file)
print(args)
print("seed", args.seed)
print("{0:20} | {1:9.3f}".format(args.file, best_acc_test))
    #scheduler.step(test_loss)
if __name__ == "__main__":
main()
